### Merging Inside Salespeople List With Revenue Report
### Dropping Duplicates by 'Opportunity'
### Renaming Columns
### Reformatting Market Names
### Writing 2 Versions to Separate CSVs

In [50]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from itertools import chain
from collections import Counter 
import openpyxl
import csv
import xlrd
import sys
import re

In [41]:
df1 = pd.read_csv('report1604062171181.csv')
df1.reset_index(drop=True)
df2 = pd.read_csv('IHM_Inside Sales_New-Term-Transfer_08202020.csv')
df2.reset_index(drop=True)
# df1.head()
# display(HTML(df2.to_html()))
# display(HTML(df1.to_html()))

,Employee Name,Email,End Date,Status,Type
0,Adrian Gillaspy,AdrianGillaspy@iheartmedia.com,8/28/18,Term,Support - Remove
1,Aimee Kemp,AimeeKemp@iheartmedia.com,7/5/17,Term,Sales
2,Alena Schlenker,AlenaSchlenker@iheartmedia.com,7/20/18,Term,Support - Remove
3,Alex Lewis,AlexLewis3@iheartmedia.com,NaN,Active,Sales
4,Alexandra Ferraro,alexandraferraro@iheartmedia.com,3/23/18,Term,Sales
...,...,...,...,...,...
171,William Lipplett,WilliamLipplett@iheartmedia.com,6/5/17,Term,Sales
172,William Radzilowsky,WilliamRadzilowsky@iheartmedia.com,7/1/20,Term,Support - Remove
173,Yineth Bolanos,YinethBolanos@iheartmedia.com,NaN,Active,Sales
174,Zachary Pollack,ZacharyPollack@iheartmedia.com,12/29/17,Term,Sales


In [42]:
df1.shape

(10664, 16)

In [43]:
# 1. Remove 'Support - Remove' from main df
# 2. Rename Sales, 'Support - Keep'
print('unique sales types: '), print(df2['Type'].unique())
print('number of inside salespeople: '), print(len(df2['Employee Name']))

unique sales types: 
['Support - Remove' 'Sales' 'Support - Keep']
number of inside salespeople: 
176


(None, None)

In [44]:
df1.columns

Index(['Opportunity: Stage', 'Opportunity: Opportunity Owner: Market Division',
       'Opportunity: Opportunity Owner: Market Area',
       'Opportunity: Opportunity Owner: Home Operating Market',
       'Opportunity: Opportunity Owner: Full Name', 'New Business',
       'Opportunity: Account Name: Account Name',
       'Opportunity: Opportunity Name', 'Adwire Project ID', 'Form Date',
       'Opportunity: Created Date', 'Opportunity: Close Date',
       'Opportunity: Gross Expected Revenue', 'Opportunity: Gross Amount',
       'Form Created on Closed Opp',
       'Opportunity: Opportunity Owner: Market Type'],
      dtype='object')

In [45]:
list = []

# this nested loop creates list of matches between the names in df2 (inside salespeople)
# and names of salespeople in df1 (revenue report)
def inside_sales_matches():
    for row in df1['Opportunity: Opportunity Owner: Full Name']:
        for name in df2['Employee Name']:
            if row == name:
                list.append(row)

                
inside_sales_matches()

# Casts the list made above to a dataframe so that it can be counted, and because I believe the code
# below will only take another dataframe for comparison
inside_sales_presence = pd.DataFrame(list, columns=['Employee Name'])

# list of unique matches between df1 and df2
inside_sales_presence_unique = np.unique(inside_sales_presence)

# produces list of matches sorted by frequency (size)
count = inside_sales_presence.groupby(['Employee Name']).size().sort_values(ascending=False)

# locates values where salesperson name in df1 matches salesperson name in df2 (unique matches list) and
# then changes the "home operating market" to "Inside Sales"
df1.loc[df1['Opportunity: Opportunity Owner: Full Name'].isin(inside_sales_presence_unique),
        'Opportunity: Opportunity Owner: Home Operating Market'] = 'Inside Sales'

df1 = df1.rename(columns = {'Opportunity: Stage': 'stage',
                           'Opportunity: Opportunity Owner: Market Division': 'market_division',
                           'Opportunity: Opportunity Owner: Market Area': 'market_area',
                           'Opportunity: Opportunity Owner: Home Operating Market': 'market',
                           'Opportunity: Opportunity Owner: Full Name': 'ae_name',
                           'New Business': 'new_business',
                           'Opportunity: Account Name: Account Name': 'account',
                           'Opportunity: Opportunity Name': 'opportunity',
                           'Adwire Project ID': 'project_id',
                           'Form Date': 'form_date',
                           'Opportunity: Created Date': 'opportunity_created_date',
                           'Opportunity: Close Date': 'opportunity_closed_date',
                           'Opportunity: Gross Expected Revenue': 'gross_expected_revenue',
                           'Opportunity: Gross Amount': 'gross_amount',
                           'Form Created on Closed Opp': 'form_created_on_closed_opp',
                           'Opportunity: Opportunity Owner: Market Type': 'market_type'})

In [54]:
re.findall('[A-Z][a-z]*', df1.market)
df1['patterns'] = df1['market'].str.findall('[A-Z][a-z]*', df1['market'])


TypeError: expected string or bytes-like object

In [46]:
df3 = df1.drop_duplicates(subset =["opportunity"])

In [47]:
print(df3.shape, df1.shape)

(9265, 16) (10664, 16)


In [48]:
# writes new dataframes to separate CSVs, resets index to col = 0
df1.to_csv('103020_duplicates.csv', index = 0)
df3.to_csv('103020_no_duplicates.csv', index = 0)

### Dataframe of No-Market AEs

In [37]:
# List of people without markets
no_market = df1[df1['market'].isnull()]
# display(HTML(df1.to_html()))


In [38]:
no_market_list = no_market.ae_name.to_list()
no_market_list = set(no_market_list)
no_market_df = pd.DataFrame(no_market_list, columns = ['name'])

In [39]:
no_market_df.to_csv('aes_no_market.csv', index=0)

In [21]:
display(HTML(df2.to_html()))

,Employee Name,Email,End Date,Status,Type
0,Adrian Gillaspy,AdrianGillaspy@iheartmedia.com,8/28/18,Term,Support - Remove
1,Aimee Kemp,AimeeKemp@iheartmedia.com,7/5/17,Term,Sales
2,Alena Schlenker,AlenaSchlenker@iheartmedia.com,7/20/18,Term,Support - Remove
3,Alex Lewis,AlexLewis3@iheartmedia.com,NaN,Active,Sales
4,Alexandra Ferraro,alexandraferraro@iheartmedia.com,3/23/18,Term,Sales
5,Alexia Harger,AlexiaHarger@iheartmedia.com,6/27/18,Term,Sales
6,Allistair Gibson,AllistairGibson@iheartmedia.com,NaN,Active,Sales
7,Amanda Hanusa,AmandaHanusa@iheartmedia.com,8/5/19,Term,Sales
8,Amanda Shannon,AmandaShannon@iheartmedia.com,5/14/19,Term,Sales
9,Amber Beilstein,AmberBeilstein@iheartmedia.com,NaN,Active,Sales


In [ ]:
d